In [11]:
a = "abcm"

a.__getitem__(slice(2, len(a)))

'cm'

In [27]:
import platform
platform.system()

'Linux'

In [19]:
import os
"/mnt/d" == os.PathLike

False

In [23]:
def a(root_folder:os.PathLike):
    pass
a(1)

In [26]:
isinstance("/mnt/d", os.PathLike | str)

True